In [1]:
import os


In [3]:
import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [4]:
#condition_match_folder불러오기
match_folder=np.loadtxt('/home/sumins/workspace/washout classification/base_data/condition_match_folder.txt',dtype=str)
match_folder=match_folder.tolist()

In [5]:
washout_slice=[]
washout_label=[]
total_slice=0
data_path='/disk1/HCC2'
for i in match_folder:    
    washout_dir = os.path.join(data_path, 'Anonymize_'+i,'washout_'+i+'.txt') 
    washout=[]
    with open(washout_dir,'r') as f:
        lines=f.readlines()
        slice=[]
        label=[]
        for line in lines:
            line=line.strip()
            washout.append(line)
        for i in washout:
            k=i.split('\t')
            total_slice+=1
            k[1]=float(k[1])
            k[1]=int(k[1])
            slice.append(int(k[0]))
            label.append(k[1])
    washout_slice.append(slice) 
    washout_label.append(label)   

In [6]:
print(len(washout_slice))
print(len(washout_label))

165
165


In [7]:
zero=0
one=0
for i in washout_label:
    for j in i:
        if j==0:
            zero+=1
        elif j==1:
            one+=1
print("True: ", one)
print("False: ", zero)
print("Total slice: ",total_slice)

True:  1379
False:  482
Total slice:  1861


In [8]:
import ast

path1='/home/sumins/workspace/washout classification/min_slice/arterial.txt'

path4='/home/sumins/workspace/washout classification/min_slice/portal.txt'

arterial=[]
portal=[]

with open (path1,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        arterial.append(ast.literal_eval(line))

with open (path4,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        portal.append(ast.literal_eval(line))

In [12]:
Train_washout_slice=washout_slice[:130]
Test_washout_slice=washout_slice[130:]
Train_washout_label=washout_label[:130]
Test_washout_label=washout_label[130:]

In [13]:
zero=0
one=0
for i in Train_washout_label:
    for j in i:
        if j==0:
            zero+=1
        elif j==1:
            one+=1
print("Train")
print("True: ", one)
print("False: ", zero)
print("Total slice: ",one+zero)

Train
True:  1108
False:  373
Total slice:  1481


In [14]:
zero=0
one=0
for i in Test_washout_label:
    for j in i:
        if j==0:
            zero+=1
        elif j==1:
            one+=1
print("Test")
print("True: ", one)
print("False: ", zero)
print("Total slice: ",one+zero)

Test
True:  271
False:  109
Total slice:  380


In [15]:

all_phase=[]
LABEL=[]
for i in range(len(Train_washout_label)):
    folder_name=match_folder[i]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()
    
    for k,j in enumerate(Train_washout_slice[i]):
        if j not in portal[i]:
            continue
        index=portal[i].index(j)
    
        a=a_data_all[:,:,arterial[i][index]]
        p=p_data_all[:,:,portal[i][index]]
        DATASET=np.stack([a,p],axis=-1)
        all_phase.append(DATASET)

        LABEL.append(Train_washout_label[i][k])
    print(i, "done")
        

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [17]:
print("Before: 1481")
print("After: ", len(all_phase))

Before: 1481
After:  1422


In [18]:
k=np.array(all_phase)
data=nib.Nifti1Image(k,None)
nib.save(data,'/home/sumins/workspace/washout classification/2phase_slice/channel4_train.nii')

In [19]:
np.savetxt("/home/sumins/workspace/washout classification/2phase_slice/label_train.txt",LABEL,fmt='%d')

In [20]:

LABELt=[]
all_phaset=[]
for i in range(len(Test_washout_label)):
    total_num=i+130
    folder_name=match_folder[total_num]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()

    
    for k,j in enumerate(Test_washout_slice[i]):
        if j not in portal[total_num]:
            continue
        index=portal[total_num].index(j)
        
        a=a_data_all[:,:,arterial[total_num][index]]
        p=p_data_all[:,:,portal[total_num][index]]
        DATASETt=np.stack([a,p],axis=-1)
        all_phaset.append(DATASETt)        

        LABELt.append(Test_washout_label[i][k])
    print(total_num, "done")
        

130 done
131 done
132 done
133 done
134 done
135 done
136 done
137 done
138 done
139 done
140 done
141 done
142 done
143 done
144 done
145 done
146 done
147 done
148 done
149 done
150 done
151 done
152 done
153 done
154 done
155 done
156 done
157 done
158 done
159 done
160 done
161 done
162 done
163 done
164 done


In [23]:
print("Before: 380")
print("After: ", len(all_phaset))

Before: 380
After:  363


In [24]:
k2=np.array(all_phaset)


In [26]:
datat=nib.Nifti1Image(k2,None)
nib.save(datat,'/home/sumins/workspace/washout classification/2phase_slice/channel4_test.nii')

In [27]:
np.savetxt("/home/sumins/workspace/washout classification/2phase_slice/label_test.txt",LABELt,fmt='%d')